# Run exaustive search for stroke arrangement using stylusengine to check scores and evaluate results

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from itertools import permutations
from math import factorial

from xmlparse import loadRef, loadGeometryBases, minXml
from score_strokes import alignStrokes

#import stylusengine

In [2]:
def getXmlScore(xml_bstring):
    score = 0#stylusengine.score_binary_string
    return score


In [3]:
f_read = ["11000"]

ref_g, ref_l, output_size = loadRef("5E7E", "Reference")
g_data, han_chars, base_data, ordered_stroke_sets = loadGeometryBases("HanBitmap", "5E7E", output_size, f_read = f_read)
g, l = g_data[0]
han_char = han_chars[0]
bases = base_data[0]
stroke_bases = ordered_stroke_sets[0]

In [4]:
heuristic_alignments = alignStrokes(g, ref_g, l, ref_l)

exhaustive_alignments = permutations(range(len(l)))


In [5]:
heuristic_xml = minXml(han_char, bases, stroke_bases, heuristic_alignments)
heuristic_score = getXmlScore(heuristic_xml)

In [6]:
def saveExhaustive(han_char, f_read):
    ref_g, ref_l, output_size = loadRef(han_char, "Reference")
    g_data, _, base_data, ordered_stroke_sets = loadGeometryBases("HanBitmap", han_char, output_size, f_read = f_read)
    for i in range(len(g_data)):
        g, l = g_data[i]
        bases = base_data[i]
        stroke_bases = ordered_stroke_sets[i]
        exhaustive_scores = np.zeros(factorial(len(l)))
        for j, p in enumerate(exhaustive_alignments):
            p_xml = minXml(han_char, bases, stroke_bases, p)
            exhaustive_scores[j] = getXmlScore(p_xml)
        print(f"Wrote exhaustive scores to {f_read[i]}_exhaust.npy")
        np.save(f"{f_read[i]}_exhaust.npy", exhaustive_scores)
